# Predicting Flight Delays: Classification Models

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [55]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [5]:
flights = pd.read_csv('../data/data-feature/flights-feature-edit.csv')

In [6]:
flights.rename(columns={'fl_date': 'date'}, inplace=True)

In [111]:
flights_df4 = flights[['op_unique_carrier', 'tail_num', 'origin', 'dest', 'arr_delay',
       'crs_elapsed_time', 'month', 'day', 'month_arr_delay_avg', 'dep_hour', 'dep_min', 'arr_hour',
       'arr_min', 'plane_score', 'route_mean_delay', 'route_median_delay', 'route_lq_delay',
       'route_uq_delay', 'route_prob_delay', 'route_mean_dep_delay',
       'route_median_dep_delay', 'route_lq_dep_delay', 'route_uq_dep_delay']]

In [66]:
def is_delay(delay):
    if delay <= 0:
        return 0
    else:
        return 1

In [112]:
flights_df4['is_delay'] = flights_df4['arr_delay'].map(is_delay)

In [113]:
flights_df4.columns

Index(['op_unique_carrier', 'tail_num', 'origin', 'dest', 'arr_delay',
       'crs_elapsed_time', 'month', 'day', 'month_arr_delay_avg', 'dep_hour',
       'dep_min', 'arr_hour', 'arr_min', 'plane_score', 'route_mean_delay',
       'route_median_delay', 'route_lq_delay', 'route_uq_delay',
       'route_prob_delay', 'route_mean_dep_delay', 'route_median_dep_delay',
       'route_lq_dep_delay', 'route_uq_dep_delay', 'is_delay'],
      dtype='object')

In [114]:
flights_df4.drop('arr_delay', axis=1, inplace=True)

In [115]:
flights_df4 = flights_df4.sample(frac=1)[:100000]

In [116]:
X = flights_df4.drop('is_delay', axis=1)
y = flights_df4['is_delay']

In [117]:
tail_num_le = LabelEncoder()
X['tail_num'] = tail_num_le.fit_transform(X['tail_num'])

airports_le = LabelEncoder()
X['origin'] = airports_le.fit_transform(X['origin'])
X['dest'] = airports_le.fit_transform(X['dest'])

op_carrier_le = LabelEncoder()
X['op_unique_carrier'] = op_carrier_le.fit_transform(X['op_unique_carrier'])

In [74]:
#dummies = pd.get_dummies(X[['dep_time_desc', 'arr_time_desc', 'orr_busy', 'arr_busy']])
#X = pd.concat([X, dummies], axis=1)

In [48]:
#X.drop(['dep_time_desc', 'arr_time_desc', 'orr_busy', 'arr_busy'], axis=1, inplace=True)

In [118]:
X_transformed = X.to_numpy()

In [119]:
y_transformed = y.to_numpy().reshape(-1,1)

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y_transformed, train_size=0.75)

In [121]:
#Gradient Boost 

gbc = GradientBoostingClassifier()

gbc.fit(X_train,y_train)

GradientBoostingClassifier()

In [122]:
gbc.score(X_test, y_test)

0.70996

In [123]:
y_pred_gbc = gbc.predict(X_test)

In [124]:
print(classification_report(y_test, y_pred_gbc))

              precision    recall  f1-score   support

           0       0.72      0.90      0.80     16247
           1       0.65      0.37      0.47      8753

    accuracy                           0.71     25000
   macro avg       0.69      0.63      0.63     25000
weighted avg       0.70      0.71      0.68     25000



In [58]:
#XGB Classifier

In [125]:
xgb_c = xgb.XGBClassifier()

In [126]:
xgb_c.fit(X_train, y_train)

[15:38:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [127]:
xgb_c.score(X_test, y_test)

0.703

In [128]:
y_pred_xgb = xgb_c.predict(X_test)

In [129]:
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.73      0.87      0.79     16247
           1       0.62      0.39      0.48      8753

    accuracy                           0.70     25000
   macro avg       0.67      0.63      0.64     25000
weighted avg       0.69      0.70      0.68     25000



In [130]:
xgb_c.feature_importances_

array([0.02436872, 0.02240103, 0.0207277 , 0.02239834, 0.02136022,
       0.02524356, 0.02498706, 0.02401539, 0.04722809, 0.0211896 ,
       0.03515382, 0.02061112, 0.02946709, 0.02210673, 0.21302576,
       0.04769379, 0.23974611, 0.04675214, 0.02386436, 0.02196564,
       0.0206302 , 0.02506358], dtype=float32)

In [110]:
X.columns

Index(['op_unique_carrier', 'tail_num', 'origin', 'dest', 'crs_elapsed_time',
       'month', 'day', 'month_arr_delay_avg', 'is_weekend', 'dep_hour',
       'dep_min', 'arr_hour', 'arr_min', 'plane_score', 'route_mean_delay',
       'route_median_delay', 'route_lq_delay', 'route_uq_delay',
       'route_prob_delay', 'route_mean_dep_delay', 'route_median_dep_delay',
       'route_lq_dep_delay', 'route_uq_dep_delay'],
      dtype='object')